In [ ]:
#when using google colab, run this code
"""
from google.colab import drive
drive.mount('/content/drive')
"""

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
os.chdir('/content/drive/MyDrive/rembg-main')

In [ ]:
import os
import torch
import torchvision
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import torch.optim as optim
import torchvision.transforms as standard_transforms

import numpy as np
import glob
import os
import gdown
os.chdir('/content/drive/MyDrive/U-2-Net-master')
from data_loader import Rescale
from data_loader import RescaleT
from data_loader import RandomCrop
from data_loader import ToTensor
from data_loader import ToTensorLab
from data_loader import SalObjDataset

from model import U2NET
from model import U2NETP


In [ ]:
import os
os.chdir('/content/drive/MyDrive/rembg-main')

0. convert pth model to onnx

In [ ]:
#run this code in ipynb cell, or run this code in cmd(in cmd, without "!")
#!pip install onnx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 96.2 MB/s eta 0:00:00


In [ ]:
import torch.onnx
import onnx

#Function to Convert model from PTH file to ONNX file
def Convert_ONNX(batch_size):

    # set the model to inference mode : (batch norm, dropout, 이동 평균 변경 방지)
    model.eval()

    dummy_input =  torch.randn(batch_size, 3,512, 512, requires_grad=True)

    # Export the model
    torch.onnx.export(model,         # model being run
         dummy_input,       # model input (or a tuple for multiple inputs)
         "./rembg/sessions/u2net_bce_itr_370000_train_0.263291_tar_0.018606.onnx",  # where to save the onnx model
         export_params=True,  # store the trained parameter weights inside the model file
         opset_version=10,    # the ONNX version to export the model to
         do_constant_folding=True,  # whether to execute constant folding for optimization
         input_names = ['modelInput'],   # the model's input names
         output_names = ['modelOutput'], # the model's output names
         dynamic_axes={'modelInput' : {0 : 'batch_size'},    # variable length axes
                                'modelOutput' : {0 : 'batch_size'}})
    print(" ")
    print('Model has been converted to ONNX')

In [ ]:
model = U2NET(3, 1)

batch_size=4 #기존 12이지만 램부족으로 모델 변경시 batch size 축소함
path = "./rembg/sessions/u2net_bce_itr_370000_train_0.263291_tar_0.018606.pth" #pth 파일
model.load_state_dict(torch.load(path))


<All keys matched successfully>

In [ ]:
Convert_ONNX(batch_size)

/usr/local/lib/python3.10/dist-packages/torch/nn/functional.py:3737: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")
/usr/local/lib/python3.10/dist-packages/torch/onnx/symbolic_helper.py:829: UserWarning: You are trying to export the model with onnx:Resize for ONNX opset version 10. This operator might cause results to not match the expected results by PyTorch.
ONNX's Upsample/Resize operator did not match Pytorch's Interpolation until opset 11. Attributes to determine how to transform the input were added in onnx:Resize in opset 11 to support Pytorch's behavior (like coordinate_transformation_mode and nearest_mode).
We recommend using opset 11 and above for models using this operator.
  warnings.warn(


============= Diagnostic Run torch.onnx.export version 2.0.1+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================

 
Model has been converted to ONNX


1. Test finetuning model

In [ ]:
#test finetuning model

#run this code in ipynb cell, or run this code in cmd(in cmd, without "!")
#!pip install pymatting
#!pip install onnxruntime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.5/52.5 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 53.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 11.8 MB/s eta 0:00:00


In [ ]:
import os
os.chdir('/content/drive/MyDrive/rembg-main') #rembg_main 모델 경로

from rembg.sessions.u2net_custom import U2netCustomSession
#from rembg import remove, new_session

from rembg.bg import remove
import cv2

In [ ]:
from typing import Any, List, Optional, Tuple, Union
from PIL import Image, ImageOps
from PIL.Image import Image as PILImage
from rembg.session_factory import new_session
from rembg.sessions.base import BaseSession
from enum import Enum
import numpy as np


class ReturnType(Enum):
    BYTES = 0
    PILLOW = 1
    NDARRAY = 2

def alpha_matting_cutout(
    img: PILImage,
    mask: PILImage,
    foreground_threshold: int,
    background_threshold: int,
    erode_structure_size: int,
) -> PILImage:
    """
    Perform alpha matting on an image using a given mask and threshold values.

    This function takes a PIL image `img` and a PIL image `mask` as input, along with
    the `foreground_threshold` and `background_threshold` values used to determine
    foreground and background pixels. The `erode_structure_size` parameter specifies
    the size of the erosion structure to be applied to the mask.

    The function returns a PIL image representing the cutout of the foreground object
    from the original image.
    """
    if img.mode == "RGBA" or img.mode == "CMYK":
        img = img.convert("RGB")

    img = np.asarray(img)
    mask = np.asarray(mask)

    is_foreground = mask > foreground_threshold
    is_background = mask < background_threshold

    structure = None
    if erode_structure_size > 0:
        structure = np.ones(
            (erode_structure_size, erode_structure_size), dtype=np.uint8
        )

    is_foreground = binary_erosion(is_foreground, structure=structure)
    is_background = binary_erosion(is_background, structure=structure, border_value=1)

    trimap = np.full(mask.shape, dtype=np.uint8, fill_value=128)
    trimap[is_foreground] = 255
    trimap[is_background] = 0

    img_normalized = img / 255.0
    trimap_normalized = trimap / 255.0

    alpha = estimate_alpha_cf(img_normalized, trimap_normalized)
    foreground = estimate_foreground_ml(img_normalized, alpha)
    cutout = stack_images(foreground, alpha)

    cutout = np.clip(cutout * 255, 0, 255).astype(np.uint8)
    cutout = Image.fromarray(cutout)

    return cutout

def naive_cutout(img: PILImage, mask: PILImage) -> PILImage:
    empty = Image.new("RGBA", (img.size), 0)
    cutout = Image.composite(img, empty, mask)
    return cutout

def putalpha_cutout(img: PILImage, mask: PILImage) -> PILImage:
    img.putalpha(mask)
    return img


def get_concat_v_multi(imgs: List[PILImage]) -> PILImage:
    pivot = imgs.pop(0)
    for im in imgs:
        pivot = get_concat_v(pivot, im)
    return pivot


def get_concat_v(img1: PILImage, img2: PILImage) -> PILImage:
    dst = Image.new("RGBA", (img1.width, img1.height + img2.height))
    dst.paste(img1, (0, 0))
    dst.paste(img2, (0, img1.height))
    return dst

def remove_hy(
    data: Union[bytes, PILImage, np.ndarray],
    alpha_matting: bool = False,
    alpha_matting_foreground_threshold: int = 240,
    alpha_matting_background_threshold: int = 10,
    alpha_matting_erode_size: int = 10,
    session: Optional[BaseSession] = None,
    only_mask: bool = False,
    post_process_mask: bool = False,
    bgcolor: Optional[Tuple[int, int, int, int]] = None,
    *args: Optional[Any],
    **kwargs: Optional[Any]
)-> Union[bytes, Image.Image, np.ndarray]:

    if isinstance(data, PILImage):
        return_type = ReturnType.PILLOW
        img = data
    elif isinstance(data, bytes):
        return_type = ReturnType.BYTES
        #img = Image.open(io.BytesIO(data))
        img=data
    elif isinstance(data, Image.Image):
            return_type = ReturnType.PILLOW
    elif isinstance(data, np.ndarray):
        return_type = ReturnType.NDARRAY
        img = Image.fromarray(data)  #array to PIL image
        #img=data #hy
    else:
        raise ValueError("Input type {} is not supported.".format(type(data)))

    putalpha = kwargs.pop("putalpha", False)


    # Fix image orientation
    #img = fix_image_orientation(img)


    if session is None:
        #session = new_session("u2net", *args, **kwargs)
        session = new_session("u2net_custom", *args, **kwargs) #세션 생성


    #img=Image.fromarray(img)


    masks = session.predict(img, *args, **kwargs)
    cutouts = []

    for mask in masks:
        if post_process_mask:
            mask = Image.fromarray(post_process(np.array(mask)))

        if only_mask:
            cutout = mask

        elif alpha_matting:
            try:
                cutout = alpha_matting_cutout(
                    img,
                    mask,
                    alpha_matting_foreground_threshold,
                    alpha_matting_background_threshold,
                    alpha_matting_erode_size,
                )
            except ValueError:
                if putalpha:
                    cutout = putalpha_cutout(img, mask)
                else:
                    cutout = naive_cutout(img, mask)
        else:
            if putalpha:
                cutout = putalpha_cutout(img, mask)
            else:
                cutout = naive_cutout(img, mask)

        cutouts.append(cutout)

    cutout = img
    if len(cutouts) > 0:
        cutout = get_concat_v_multi(cutouts)

    if bgcolor is not None and not only_mask:
        cutout = apply_background_color(cutout, bgcolor)

    if ReturnType.PILLOW == return_type:
        return cutout

    if ReturnType.NDARRAY == return_type:
        return np.asarray(cutout)

    bio = io.BytesIO()
    cutout.save(bio, "PNG")
    bio.seek(0)

    return bio.read()

In [ ]:
#define model path
modelpath= "/content/drive/MyDrive/rembg-main/rembg/sessions/u2net_bce_itr_370000_train_0.263291_tar_0.018606.onnx" #put finetuned model path

In [ ]:
#Processing images in input folder, without alpha matting
"""
import onnxruntime as ort
import io
from pathlib import Path

#create custom session
model_path = modelpath
sess_opts = ort.SessionOptions()
custom_session = U2netCustomSession("temp_u2net_custom", sess_opts, model_path=model_path)

for file in Path('/content/drive/MyDrive/rembg-main/ft_test_input').glob('*.png'):
    input_path = str(file)
    output_path = str('/content/drive/MyDrive/rembg-main/ft_test_output/' + file.name)  # Use the name of the input file for the output

    with open(input_path, 'rb') as i:
        with open(output_path, 'wb') as o:
            input = i.read()
            # Convert the input data (bytes) to a PIL Image object
            input = Image.open(io.BytesIO(input))

            output = remove_hy(input, session=custom_session)
            by = io.BytesIO()
            output.save(by, format='png')
            o.write(by.getvalue()) # Write the processed image data to the output file
            """

In [ ]:
#Processing images in input folder, this time, using alpha matting
import onnxruntime as ort
import io
from pathlib import Path
import io
from enum import Enum
from typing import Any, List, Optional, Tuple, Union

import numpy as np
from cv2 import (
    BORDER_DEFAULT,
    MORPH_ELLIPSE,
    MORPH_OPEN,
    GaussianBlur,
    getStructuringElement,
    morphologyEx,
)
from PIL import Image, ImageOps
from PIL.Image import Image as PILImage
from pymatting.alpha.estimate_alpha_cf import estimate_alpha_cf
from pymatting.foreground.estimate_foreground_ml import estimate_foreground_ml
from pymatting.util.util import stack_images
from scipy.ndimage import binary_erosion


#create custom session
model_path = modelpath
sess_opts = ort.SessionOptions()
custom_session = U2netCustomSession("temp_u2net_custom", sess_opts, model_path=model_path)

for file in Path('/content/drive/MyDrive/rembg-main/ft_test_input').glob('*.png'):
    input_path = str(file)
    output_path = str('/content/drive/MyDrive/rembg-main/ft_test_output_alpha/' + file.name)  # Use the name of the input file for the output

    with open(input_path, 'rb') as i:
        with open(output_path, 'wb') as o:
            input = i.read()
            # Convert the input data (bytes) to a PIL Image object
            input = Image.open(io.BytesIO(input))

            output = remove_hy(input, session=custom_session,alpha_matting=True )
            by = io.BytesIO()
            output.save(by, format='png')
            o.write(by.getvalue()) # Write the processed image data to the output file

PERFORMANCE WARNING:
Thresholded incomplete Cholesky decomposition failed due to insufficient positive-definiteness of matrix A with parameters:
    discard_threshold = 1.000000e-04
    shift = 0.000000e+00
Try decreasing discard_threshold or start with a larger shift

PERFORMANCE WARNING:
Thresholded incomplete Cholesky decomposition failed due to insufficient positive-definiteness of matrix A with parameters:
    discard_threshold = 1.000000e-04
    shift = 0.000000e+00
Try decreasing discard_threshold or start with a larger shift

PERFORMANCE WARNING:
Thresholded incomplete Cholesky decomposition failed due to insufficient positive-definiteness of matrix A with parameters:
    discard_threshold = 1.000000e-04
    shift = 0.000000e+00
Try decreasing discard_threshold or start with a larger shift

PERFORMANCE WARNING:
Thresholded incomplete Cholesky decomposition failed due to insufficient positive-definiteness of matrix A with parameters:
    discard_threshold = 1.000000e-04
    shif